# Image Deraining Project

### Imports

In [ ]:
# !pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu117
# !pip3 install numpy scipy tqdm opencv-python pandas scikit-learn matplotlib

In [1]:
# Python Packages
from torch.utils.data import DataLoader

import warnings
import torch
import os

# Local Modules
from utilities.utils import train_model, plot_loss
from utilities.datasets import SID_dataset, SID_dataset_mini
from models.modules import PReNet, PReNet_r, PReDecoderNet, \
    PReLSTMNet, PReComboNet, PReHAYULANet, PReAENet, PReAEATNet, EnDecoNet


In [ ]:
warnings.filterwarnings('ignore')
torch.manual_seed(42)

DEVICE = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')


IM_SIZE = 128

In [ ]:
input_path = "data/rain_streak/training/rainy_image/"
label_path = "data/rain_streak/training/ground_truth/"

valid_input_path = "data/rain_streak/validation/rainy_image/"
valid_label_path = "data/rain_streak/validation/ground_truth/"

batch_size = 4

dataset_train = SID_dataset_mini(input_path, label_path, IM_SIZE)
dataset_valid = SID_dataset(valid_input_path, valid_label_path, IM_SIZE)
train_loader = DataLoader(
    dataset_train, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(
    dataset_valid, batch_size=batch_size, shuffle=True, pin_memory=True)


In [ ]:
net1 = PReDecoderNet().to(DEVICE)
net2 = PReLSTMNet().to(DEVICE)
net3 = PReComboNet().to(DEVICE)
net4 = PReHAYULANet().to(DEVICE)
net5 = PReAENet(im_size=IM_SIZE).to(DEVICE)
net6 = PReAEATNet(im_size=IM_SIZE).to(DEVICE)
net7 = EnDecoNet().to(DEVICE)
net8 = PReNet().to(DEVICE)
net9 = PReNet_r().to(DEVICE)

p_net1 = sum(p.numel() for p in net1.parameters())
p_net2 = sum(p.numel() for p in net2.parameters())
p_net3 = sum(p.numel() for p in net3.parameters())
p_net4 = sum(p.numel() for p in net4.parameters())
p_net5 = sum(p.numel() for p in net5.parameters())
p_net6 = sum(p.numel() for p in net6.parameters())
p_net7 = sum(p.numel() for p in net7.parameters())
p_net8 = sum(p.numel() for p in net8.parameters())
p_net9 = sum(p.numel() for p in net9.parameters())

print(f'PRettentiveNet(Deconv):\n\t\t\t\t{p_net1}', end='\n\n\n')
print(f'PRettentiveNet(multi-LSTM):\n\t\t\t\t{p_net2}', end='\n\n\n')
print(f'PRettentiveNet(combo):\n\t\t\t\t{p_net3}', end='\n\n\n')
print(f'PRettentiveNet(HAYULA):\n\t\t\t\t{p_net4}', end='\n\n\n')
print(f'PReAENet:\n\t\t\t\t{p_net5}', end='\n\n\n')
print(f'PReAEATNet:\n\t\t\t\t{p_net6}', end='\n\n\n')
print(f'EnDecoNet:\n\t\t\t\t{p_net7}', end='\n\n\n')
print(f'PReNet:\n\t\t\t\t{p_net8}', end='\n\n\n')
print(f'PReNet_r:\n\t\t\t\t{p_net9}', end='\n\n\n')


In [ ]:
models = [
    # [net9, 'PReNet_r'],
    # [net8, 'PReNet'],
    # [net5, 'PRettentiveNet(AutoEncoder)'],
    # [net6, 'PRettentiveNet(AutoEncoderplus)'],
    [net7, 'EnDecoNet'],
    # [net1, 'PRettentiveNet(DECONV)'],
    # [net2, 'PRettentiveNet(MULTI-LSTM)'],
    # [net3, 'PRettentiveNet(COMBO)'],
    # [net4, 'PRettentiveNet(HAYULA)'],
]

train_dict = {
    'train_loader': train_loader,
    'valid_loader': valid_loader,
    'device': DEVICE,
    'epoch': 100
}


In [ ]:
results = []
for i, model in enumerate(models):
    if not os.path.isdir(f'trained_models/{model[1]}'):
        os.mkdir(f'trained_models/{model[1]}')
    if not os.path.isdir(f'trained_models/{model[1]}/Low Data'):
        os.mkdir(f'trained_models/{model[1]}/Low Data')
    print(f'Training {model[1]}')
    result = train_model(model, **train_dict)
    result = [result, model[1]]
    results.append(result)


In [ ]:
for result in results:
    print(result[1])
    plot_loss(**result[0])


In [ ]:
import os
if not os.path.isdir('mamad'):
    os.mkdir('mamad')
if not os.path.isdir('mamad/mamads'):
    os.mkdir('mamad/mamads')